# TabNet CODE

### Introduce TabNET
---
* 고스트 배치 정규화 (GBN)

* Sparsemax

* 


### code_source
---
* TabNet Torch code: https://ichi.pro/ko/pytorcheseo-tabnet-guhyeon-277727554318969
* Sparsemax code: https://github.com/gokceneraslan/SparseMax.torch
* paper: https://arxiv.org/pdf/1908.07442v4.pdf
* pytorch-TabNet1: https://pypi.org/project/pytorch-tabnet/
* pytorch-TabNet2: https://wsshin.tistory.com/5
* pytorch-TabNet-Regressor: https://www.kaggle.com/rapela/tps-02-21-tabnet-regressor


### Parameter Tuning
---
* Source: https://dreamquark-ai.github.io/tabnet/generated_docs/README.html#model-parameters

#### Model parameters

1. n_d: int(default = 8)
    - Width of the decision prediction layer. Bigger values gives more capacity to the model with the risk of overfitting. Values typically range from 8 to 64.
<br/> 
2. n_a: int(default = 8)
    - Width of the attention embedding for each mask. According to the paper n_d=n_a is usually a good choice. (default=8)
<br/> 
3. n_steps: int(default = 3)
    - Number of steps in the architecture (usually between 3 and 10)
<br/> 
4. gamma: float (default = 1.3)
    - This is the coefficient for feature reusage in the masks. A value close to 1 will make mask selection least correlated between layers. Values range from 1.0 to 2.0.
<br/> 
5. cat_idxs: list of int (default=[] - Mandatory for embeddings)
    - List of categorical features indices.
<br/> 
6. cat_dims: list of int (default=[] - Mandatory for embeddings)
    - List of categorical features number of modalities (number of unique values for a categorical feature) /!\ no new modalities can be predicted
<br/> 
7. cat_emb_dim : list of int (optional)
    - List of embeddings size for each categorical features. (default =1)
<br/> 
8. n_independent : int (default=2)
    - Number of independent Gated Linear Units layers at each step. Usual values range from 1 to 5.
<br/> 
9. n_shared : int (default=2)
    - Number of shared Gated Linear Units at each step Usual values range from 1 to 5
<br/> 
10. epsilon : float (default 1e-15)
    - Should be left untouched
<br/> 
11. seed : int (default=0)
    - Random seed for reproducibility
<br/> 
12. momentum : float
    - Momentum for batch normalization, typically ranges from 0.01 to 0.4 (default=0.02)
<br/> 
13. clip_value : float (default None)
    - If a float is given this will clip the gradient at clip_value.
<br/> 
14. lambda_sparse : float (default = 1e-3)
    - This is the extra sparsity loss coefficient as proposed in the original paper. The bigger this coefficient is, the sparser your model will be in terms of feature selection. Depending on the difficulty of your problem, reducing this value could help.
<br/>
15. optimizer_fn : torch.optim (default=torch.optim.Adam)
    - Pytorch optimizer function
<br/>     
16. optimizer_params: dict (default=dict(lr=2e-2))
    - Parameters compatible with optimizer_fn used initialize the optimizer. Since we have Adam as our default optimizer, we use this to define the initial learning rate used for training. As mentionned in the original paper, a large initial learning of 0.02 with decay is a good option.
<br/> 
17. scheduler_fn : torch.optim.lr_scheduler (default=None)
    - Pytorch Scheduler to change learning rates during training.
<br/> 
18. model_name : str (default = ‘DreamQuarkTabNet’)
    - Name of the model used for saving in disk, you can customize this to easily retrieve and reuse your trained models.
<br/> 
19. saving_path : str (default = ‘./’)
    - Path defining where to save models.
<br/> 
20. verbose : int (default=1)
    - Verbosity for notebooks plots, set to 1 to see every epoch, 0 to get None.
<br/> 
21. device_name : str (default=’auto’) 
    - ‘cpu’ for cpu training, ‘gpu’ for gpu training, ‘auto’ to automatically detect gpu.
<br/> 
22. mask_type: str (default=’sparsemax’) 
    - Either “sparsemax” or “entmax” : this is the masking function to use for selecting features
<br/> 
<br/> 

#### Fit parameters
1. X_train : np.array
<br/> 
2. y_train : np.array
<br/> 
3. eval_set: list of tuple
<br/> 
4. eval_name: list of str
<br/> 
5. eval_metric : list of str
<br/> 
6. max_epochs : int (default = 200)
<br/> 
7. patience : int (default = 15)
<br/> 
8. weights : int or dict (default=0)
    -  Only for TabNetClassifier Sampling parameter 0 : no sampling 1 : automated sampling with inverse class occurrences dict : keys are classes, values are weights for each class
<br/> 
9. loss_fn : torch.loss or list of torch.loss
    - Loss function for training (default to mse for regression and cross entropy for classification) When using TabNetMultiTaskClassifier you can set a list of same length as number of tasks, each task will be assigned its own loss function
<br/> 
10. batch_size : int (default=1024)
    - Number of examples per batch, large batch sizes are recommended
<br/>     
11. virtual_batch_size : int (default=128)
    - Size of the mini batches used for “Ghost Batch Normalization”. /!\ virtual_batch_size should divide batch_size
<br/> 
12. num_workers : int (default=0)
    - Number or workers used in torch.utils.data.Dataloader
<br/> 
13. drop_last : bool (default=False)
    - Whether to drop last batch if not complete during training
<br/> 
14. callbacks : list of callback function
    - List of custom callbacks
<br/> 
15. pretraining_ratio : float
      - /!\ TabNetPretrainer Only : Percentage of input features to mask during pretraining.
      - Should be between 0 and 1. The bigger the harder the reconstruction task is.

In [2]:
import warnings

warnings.filterwarnings( 'ignore' )

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from pytorch_tabnet.tab_model import TabNetRegressor

from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error

import random

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
SEED = 42

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(SEED)


train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")

x_data = train_data.loc[:, 'f0':'f99']
y_data = train_data.loc[:, 'loss']

x_train, x_test, y_train, y_test=train_test_split(x_data,
                                                  y_data,
                                                  test_size=0.3,   #전체 중 20%를 테스트용으로 분할
                                                                   #나머지 80%는 훈련용
                                                  shuffle=True,    #무작위로 섞어서 추출
                                                  random_state=SEED) #무작위 추출 시 일정한 기준으로
x_val, x_test, y_val, y_test = train_test_split(x_test,
                                                y_test,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=SEED)

y_train = y_train.values
y_val = y_val.values


for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(x_train[c].values) + list(x_test[c].values))

        x_train[c] = lbl.transform(x_train[c].values)
        x_test[c] = lbl.transform(x_test[c].values)

columns = x_test.columns

In [3]:
def Optuna_TabNet(trial):
    N_D = 16
    N_A = N_D
    N_STEPS = trial.suggest_int("N_STEPS", 3, 5)
    GAMMA = trial.suggest_uniform("GAMMA", 1.0, 2.0)
    N_INDEPENDENT = trial.suggest_int("N_INDEPENDENT", 1, 3)
    N_SHARED = trial.suggest_int("N_SHARED", 1, 3)
    LAMBDA_SPARSE =  trial.suggest_uniform("LAMBDA_SPARSE", 0, 1e-2)
    OPT_LR = trial.suggest_categorical('OPT_LR', [1e-1 ,5e-2, 1e-2, 1e-3, 1e-4])
    OPT_WEIGHT_DECAY = trial.suggest_categorical('OPT_WEIGHT_DECAY', [1e-8, 1e-6, 1e-5, 1e-4, 1e-3])
    OPT_MOMENTUM = trial.suggest_uniform("OPT_MOMENTUM", 0.01, 0.4)
#     MASK_TYPE = trial.suggest_categorical('MASK_TYPE',  ["sparsemax", "entmax"])
    MASK_TYPE = "entmax"
    
    SCHEDULER_MIN_LR = 1e-6
    SCHEDULER_FACTOR = 0.9
    
    tabnet_params = dict(n_d=N_D, 
                         n_a=N_A, 
                         n_steps=N_STEPS, 
                         gamma=GAMMA,
                         n_independent = N_INDEPENDENT,
                         n_shared = N_SHARED,
                         lambda_sparse=LAMBDA_SPARSE, 
                         optimizer_fn=torch.optim.SGD,
                         optimizer_params=dict(lr=OPT_LR, 
                                               weight_decay=OPT_WEIGHT_DECAY, 
                                               momentum=OPT_MOMENTUM),
                         mask_type=MASK_TYPE,
                         scheduler_params=dict(mode="min",
                                               patience=20,
                                               min_lr=SCHEDULER_MIN_LR,
                                               factor=SCHEDULER_FACTOR,),
                         scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                         verbose=1,
                         seed=SEED
                         )
    print(tabnet_params)
    
    return TabNetRegressor(**tabnet_params)

def objective(trial):
    
    MAX_EPOCH = trial.suggest_categorical("MAX_EPOCH", [1000, 3000, 5000])
    BATCH_SIZE = 512
    
    train_df, val_df = x_train.iloc[:][columns], x_val.iloc[:][columns]
    
    train_df = train_df.to_numpy()
    train_target = y_train.reshape(-1, 1)
    
    val_df = val_df.to_numpy()
    val_target = y_val.reshape(-1, 1)
    
    model = Optuna_TabNet(trial)
    
    #with pruning
    for step in range(5):
        model.fit(X_train=train_df,
              y_train=train_target,
              max_epochs=3,
              batch_size=BATCH_SIZE,
              num_workers= 4 * torch.cuda.device_count(),
              drop_last=False)
        test_score = mean_squared_error(model.predict(x_test.to_numpy()), y_test)
        
        trial.report(test_score, step)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        
    model.fit(X_train=train_df,
              y_train=train_target,
              eval_set=[(val_df, val_target)],
              eval_name = ["val"],
              eval_metric = ['mse'],
              max_epochs=MAX_EPOCH,
              patience=10, 
              batch_size=BATCH_SIZE,
              num_workers= 4 * torch.cuda.device_count(),
              drop_last=False)

    score = mean_squared_error(model.predict(x_test.to_numpy()), y_test)
    print(score)
    
    return score

In [ ]:
TRIAL_NUM = 100

study = optuna.create_study(direction='minimize',
                            sampler=TPESampler(),
                            pruner=optuna.pruners.MedianPruner())
study.optimize(lambda trial : objective(trial), n_trials=TRIAL_NUM)

joblib.dump(study, "study_TABNET16_2.pkl")

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])



print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


# print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))
# best_param = study.best_trial.params

[I 2021-08-23 17:27:53,892] A new study created in memory with name: no-name-c9dd28b4-633e-4b81-9a84-9afc8663c7fa


{'n_d': 16, 'n_a': 16, 'n_steps': 5, 'gamma': 1.958297534323594, 'n_independent': 2, 'n_shared': 1, 'lambda_sparse': 0.0005835697725547651, 'optimizer_fn': <class 'torch.optim.sgd.SGD'>, 'optimizer_params': {'lr': 0.01, 'weight_decay': 1e-06, 'momentum': 0.38855612095763653}, 'mask_type': 'entmax', 'scheduler_params': {'mode': 'min', 'patience': 20, 'min_lr': 1e-06, 'factor': 0.9}, 'scheduler_fn': <class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>, 'verbose': 1, 'seed': 42}
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 113.39919|  0:00:24s
epoch 1  | loss: 93.44884|  0:00:48s
epoch 2  | loss: 83.28976|  0:01:12s
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 78.22799|  0:00:24s
epoch 1  | loss: 75.33061|  0:00:48s
epoch 2  | loss: 73.827  |  0:01:12s
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 72.66759|  0:00:24s
epoch 1  | loss: 72.11

epoch 121| loss: 65.00466| val_mse: 64.95821|  0:51:49s
epoch 122| loss: 65.05924| val_mse: 64.89676|  0:52:14s
epoch 123| loss: 64.87992| val_mse: 64.86951|  0:52:40s
epoch 124| loss: 64.94381| val_mse: 64.90343|  0:53:05s
epoch 125| loss: 64.96312| val_mse: 64.85564|  0:53:30s
epoch 126| loss: 64.92979| val_mse: 64.81557|  0:53:55s
epoch 127| loss: 64.90713| val_mse: 64.81686|  0:54:20s
epoch 128| loss: 64.9406 | val_mse: 64.81484|  0:54:45s
epoch 129| loss: 64.91181| val_mse: 64.7639 |  0:55:10s
epoch 130| loss: 64.78753| val_mse: 64.69126|  0:55:35s
epoch 131| loss: 64.80199| val_mse: 64.65711|  0:56:00s
epoch 132| loss: 64.90514| val_mse: 64.66295|  0:56:25s
epoch 133| loss: 64.79712| val_mse: 64.65936|  0:56:50s
epoch 134| loss: 64.74765| val_mse: 64.51369|  0:57:15s
epoch 135| loss: 64.68934| val_mse: 64.62234|  0:57:41s
epoch 136| loss: 64.73228| val_mse: 64.50825|  0:58:06s
epoch 137| loss: 64.66332| val_mse: 64.64063|  0:58:31s
epoch 138| loss: 64.59025| val_mse: 64.54278|  0

epoch 268| loss: 63.34352| val_mse: 63.15535|  1:53:35s
epoch 269| loss: 63.32034| val_mse: 63.1798 |  1:54:01s
epoch 270| loss: 63.30631| val_mse: 63.15886|  1:54:27s
epoch 271| loss: 63.31644| val_mse: 63.1555 |  1:54:52s
epoch 272| loss: 63.33259| val_mse: 63.1455 |  1:55:18s
epoch 273| loss: 63.27819| val_mse: 63.16482|  1:55:44s

Early stopping occurred at epoch 273 with best_epoch = 263 and best_val_mse = 63.13254
Best weights from best epoch are automatically used!


[I 2021-08-23 19:30:47,058] Trial 0 finished with value: 63.86604804999591 and parameters: {'MAX_EPOCH': 3000, 'N_STEPS': 5, 'GAMMA': 1.958297534323594, 'N_INDEPENDENT': 2, 'N_SHARED': 1, 'LAMBDA_SPARSE': 0.0005835697725547651, 'OPT_LR': 0.01, 'OPT_WEIGHT_DECAY': 1e-06, 'OPT_MOMENTUM': 0.38855612095763653}. Best is trial 0 with value: 63.86604804999591.


...

epoch 121| loss: 63.83717| val_mse: 63.67811|  0:28:52s
epoch 122| loss: 63.79435| val_mse: 63.68439|  0:29:05s
epoch 123| loss: 63.81703| val_mse: 63.67145|  0:29:19s
epoch 124| loss: 63.79658| val_mse: 63.68815|  0:29:33s
epoch 125| loss: 63.71506| val_mse: 63.67262|  0:29:47s
epoch 126| loss: 63.72335| val_mse: 63.63227|  0:30:01s
epoch 127| loss: 63.67111| val_mse: 63.60761|  0:30:15s
epoch 128| loss: 63.71891| val_mse: 63.6286 |  0:30:29s
epoch 129| loss: 63.71612| val_mse: 63.59725|  0:30:43s
epoch 130| loss: 63.73874| val_mse: 63.60611|  0:30:57s
epoch 131| loss: 63.68907| val_mse: 63.52208|  0:31:11s
epoch 132| loss: 63.66834| val_mse: 63.5446 |  0:31:25s
epoch 133| loss: 63.70216| val_mse: 63.54725|  0:31:40s
epoch 134| loss: 63.59991| val_mse: 63.50119|  0:31:54s
epoch 135| loss: 63.62388| val_mse: 63.50319|  0:32:08s
epoch 136| loss: 63.57818| val_mse: 63.5028 |  0:32:22s
epoch 137| loss: 63.56292| val_mse: 63.45344|  0:32:36s
epoch 138| loss: 63.61981| val_mse: 63.46022|  0

epoch 268| loss: 62.98932| val_mse: 62.89773|  1:03:44s
epoch 269| loss: 62.98962| val_mse: 62.90235|  1:03:59s
epoch 270| loss: 62.97992| val_mse: 62.89533|  1:04:13s
epoch 271| loss: 62.95244| val_mse: 62.9031 |  1:04:28s
epoch 272| loss: 62.95272| val_mse: 62.88817|  1:04:42s
epoch 273| loss: 62.96172| val_mse: 62.89632|  1:04:57s
epoch 274| loss: 62.95989| val_mse: 62.88649|  1:05:11s
epoch 275| loss: 62.98253| val_mse: 62.87409|  1:05:25s
epoch 276| loss: 62.95285| val_mse: 62.89662|  1:05:39s
epoch 277| loss: 62.9801 | val_mse: 62.88087|  1:05:54s
epoch 278| loss: 62.94051| val_mse: 62.87687|  1:06:09s
epoch 279| loss: 62.95909| val_mse: 62.88646|  1:06:23s
epoch 280| loss: 62.96047| val_mse: 62.88659|  1:06:38s
epoch 281| loss: 62.93001| val_mse: 62.88446|  1:06:52s
epoch 282| loss: 62.93806| val_mse: 62.8889 |  1:07:06s
epoch 283| loss: 62.9257 | val_mse: 62.88326|  1:07:20s
epoch 284| loss: 62.94919| val_mse: 62.89322|  1:07:35s
epoch 285| loss: 62.92942| val_mse: 62.89065|  1

[I 2021-08-23 20:42:38,044] Trial 1 finished with value: 63.66390757345737 and parameters: {'MAX_EPOCH': 5000, 'N_STEPS': 3, 'GAMMA': 1.4461063105212684, 'N_INDEPENDENT': 1, 'N_SHARED': 1, 'LAMBDA_SPARSE': 0.006074077172986856, 'OPT_LR': 0.001, 'OPT_WEIGHT_DECAY': 0.001, 'OPT_MOMENTUM': 0.22458437755775237}. Best is trial 1 with value: 63.66390757345737.


63.66390757345737
{'n_d': 16, 'n_a': 16, 'n_steps': 5, 'gamma': 1.8866278723494483, 'n_independent': 3, 'n_shared': 2, 'lambda_sparse': 0.00530752821947752, 'optimizer_fn': <class 'torch.optim.sgd.SGD'>, 'optimizer_params': {'lr': 0.0001, 'weight_decay': 0.0001, 'momentum': 0.21210871782874902}, 'mask_type': 'entmax', 'scheduler_params': {'mode': 'min', 'patience': 20, 'min_lr': 1e-06, 'factor': 0.9}, 'scheduler_fn': <class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>, 'verbose': 1, 'seed': 42}
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 123.56118|  0:00:30s
epoch 1  | loss: 123.427 |  0:01:00s
epoch 2  | loss: 123.25354|  0:01:31s
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 123.21429|  0:00:30s
epoch 1  | loss: 123.11383|  0:01:00s
epoch 2  | loss: 123.04739|  0:01:31s
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 122.93105|  0:00:3

epoch 117| loss: 112.0039| val_mse: 110.24121|  1:06:02s
epoch 118| loss: 111.84073| val_mse: 110.0841|  1:06:35s
epoch 119| loss: 111.81557| val_mse: 110.35288|  1:07:08s
epoch 120| loss: 111.71866| val_mse: 110.45695|  1:07:41s
epoch 121| loss: 111.63105| val_mse: 110.26782|  1:08:14s
epoch 122| loss: 111.51516| val_mse: 110.0462|  1:08:48s
epoch 123| loss: 111.43489| val_mse: 110.12043|  1:09:21s
epoch 124| loss: 111.31692| val_mse: 109.7732|  1:09:55s
epoch 125| loss: 111.31433| val_mse: 109.86336|  1:10:28s
epoch 126| loss: 111.23994| val_mse: 109.58563|  1:11:02s
epoch 127| loss: 111.15095| val_mse: 110.11457|  1:11:35s
epoch 128| loss: 110.98434| val_mse: 109.26739|  1:12:09s
epoch 129| loss: 110.95449| val_mse: 109.80354|  1:12:41s
epoch 130| loss: 110.86259| val_mse: 109.55516|  1:13:15s
epoch 131| loss: 110.84559| val_mse: 109.42321|  1:13:48s
epoch 132| loss: 110.73862| val_mse: 109.27628|  1:14:21s
epoch 133| loss: 110.63484| val_mse: 109.30906|  1:14:54s
epoch 134| loss: 1

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
train_oof = np.zeros((len(x_train)))
test_preds = 0

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(x_train, y_train))):

    print(f'Fold {f}')
    train_df, val_df = x_train.iloc[train_ind][columns], x_train.iloc[val_ind][columns]

    train_target, val_target = y_train[train_ind], y_train[val_ind]

    print(train_df.shape, train_target.shape)
    print(val_df.shape, val_target.shape)

    train_target=train_target.reshape(-1,1)
    val_target=val_target.reshape(-1,1)

    train_df      = train_df.to_numpy()
    train_target      = train_target.reshape(-1, 1)

    val_df = val_df.to_numpy()
    val_target = val_target.reshape(-1, 1)

    model = TabNetRegressor(**tabnet_params)

    model.fit(X_train=train_df,
              y_train=train_target,
              eval_set=[(val_df, val_target)],
              eval_name = ["val"],
              eval_metric = ['mse'],#["logits_ll"],
              max_epochs=MAX_EPOCH, #20
              patience=20, batch_size=BATCH_SIZE,
              drop_last=False)#,

    temp_oof = model.predict(val_df)
    train_oof[val_ind] = temp_oof.reshape(-1)     
    print(mean_squared_error(temp_oof, val_target, squared=False))
    
    temp_test = model.predict(x_test.to_numpy())
    test_preds += temp_test/NUM_FOLDS

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
       'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30',
       'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40',
       'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50',
       'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60',
       'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70',
       'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90',
       'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'],
      dtype='object')


In [ ]:
# XG-Boost Score: 61.34741620778698

In [22]:
print('#### fold #########',np.sqrt(mean_squared_error(y_test, test_preds)),mean_squared_error(y_test, test_preds))

#### fold ######### 7.893137848830042 62.30162510063334


In [24]:
np.save('TabNet_ytest.npy', test_preds)

In [20]:
from pandas import Series, DataFrame

raw_data = {'id': [ i for i in range(250000,400000)],
            'loss':  }

data = DataFrame(raw_data)
data.set_index('id', inplace=True)
print(data)
data.to_csv("submission.csv", mode='w')

ValueError: All arrays must be of the same length